In [1]:
! pip install chromadb

## 1. Run Chroma

In [2]:
# Ephemeral Client (dữ liệu sẽ chỉ được lưu trữ trong thờ gian session chạy)
import chromadb
client = chromadb.Client()

In [2]:
# Persistent Client (dữ liệu sẽ được lưu lại và load lên khi cần)
import chromadb
client = chromadb.PersistentClient(path="./chroma_db")

## 2. Collection

In [3]:
from datetime import datetime

collection1 = client.create_collection(
    name="my_collection1",
    # embedding_function=emb_fn, # nếu không có tham số này Chroma sẽ tự động sử dụng SBert để embedding
    metadata={
        "description": "my first Chroma collection",
        "created": str(datetime.now())
    }
)


In [5]:
# một vài method hữu ích của Collection
print(collection1.peek()) # trả về 10 item đầu tiên
print(collection1.count()) # đếm số lượng item trong Collection
# collection1.modify(name="new_name") # rename

{'ids': [], 'embeddings': array([], dtype=float64), 'documents': [], 'uris': None, 'data': None, 'metadatas': [], 'included': [<IncludeEnum.embeddings: 'embeddings'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
0


### Configuring Chroma Collections
--------------------------
* `hnsw:space`: Định nghĩa hàm tính khoảng cách giữa các embeddings, bao gồm các options: `l2`, `ip`, `cosine`
* `hnsw:construction_ef`: xác định số lượng các nút (hoặc vector ứng viên) được xem xét trong quá trình tìm kiếm (query time)
* `hnsw:M`: (tùy chọn) cấu hình tham số trong quá trình xây dựng chỉ số
* `hnsw:num_threads`: (tùy chọn) số lượng liên kết tối đa mỗi nút

In [6]:
collection2 = client.create_collection(
    name="my_collection2",
    # embedding_function=emb_fn,
    metadata={
        "hnsw:space": "cosine",
        "hnsw:search_ef": 100
    }
)

### Adding Data to Chroma Collections

In [10]:
collection2.add(
    documents=["lorem ipsum...", "doc2", "doc3"],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 103MiB/s]


Nếu Chroma được truyền một danh sách các tài liệu, nó sẽ tự động mã hóa và nhúng chúng bằng hàm nhúng của `Collection` (mặc định sẽ được sử dụng SBert nếu không có hàm nhúng nào được cung cấp khi tạo bộ sưu tập). Chroma cũng sẽ lưu trữ chính các tài liệu đó. Nếu các tài liệu quá lớn để nhúng bằng hàm nhúng đã chọn, một ngoại lệ sẽ được đưa ra.

Mỗi tài liệu phải có một `id` liên kết duy nhất. Nếu `.add` cùng một ID hai lần sẽ chỉ dẫn đến giá trị ban đầu được lưu trữ. Có thể cung cấp danh sách tùy chọn các từ điển siêu dữ liệu cho mỗi tài liệu để lưu trữ thông tin bổ sung và cho phép lọc.

Ngoài ra, bạn có thể cung cấp danh sách các `embeddings` của tài liệu trực tiếp và Chroma sẽ lưu trữ các tài liệu mà không nhúng chúng nữa.Nếu các `embeddings` được cung cấp không có cùng kích thước với bộ sưu tập, một ngoại lệ sẽ được đưa ra.

In [13]:
collection1.add(
    documents=["doc1", "doc2", "doc3"],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    ids=["id1", "id2", "id3"]
)

### Updating Data in Chroma Collections

In [15]:
# nếu update mà id không tồn tại sẽ báo lỗi
collection1.update(
    ids=["id1", "id2", "id3"],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    documents=["doc1", "doc2", "doc3"],
)


In [34]:
# sử dụng upsert: nếu tồn tại rồi sẽ cập nhật(update) còn chưa tồn tại thì thêm mới(add)
collection1.upsert(
    ids=["id1", "id2", "id3"],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2]],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}],
    documents=["doc1", "doc2", "doc3"],
)

### Deleting Data from Chroma Collections
`.delete` also supports the `where` filter

In [35]:
# có thể dùng id hoặc where để xóa, nếu chỉ có where thì nó sẽ xóa toàn
collection1.delete(
    ids=["id1"],
    where = {"chapter": "3"}
)

In [36]:
collection1.count()

2

## Query Collection

In [57]:
collection2.query(
    query_texts=["doc"],
    n_results=2,
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"},
    include= ['documents']
)

{'ids': [['id2', 'id3']],
 'embeddings': None,
 'documents': [['doc2', 'doc3']],
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': None,
 'included': [<IncludeEnum.documents: 'documents'>]}

### Custom Embedding Functions

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings

class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        return embeddings
